# 🔄 RDD Operations & Transformations

**Phase 2: RDD Mastery - Core Operations & Transformations**

**Prerequisites**: [01_fundamentals/FirstSparkApp.ipynb](../01_fundamentals/FirstSparkApp.ipynb)

**Estimated time**: 45-60 minutes

---

## 🎯 Learning Goals

By the end of this notebook, you'll be able to:
- ✅ Master essential RDD transformations (map, filter, flatMap)
- ✅ Understand lazy evaluation and execution flow
- ✅ Perform aggregation operations (reduce, fold, aggregate)
- ✅ Use key-value RDD operations (groupByKey, reduceByKey)
- ✅ Apply set operations and joins
- ✅ Optimize RDD operations for performance

---

## 📋 Table of Contents

1. [RDD Transformations](#transformations)
2. [Aggregation Operations](#aggregation)
3. [Key-Value Operations](#keyvalue)
4. [Set Operations](#setops)
5. [Performance Patterns](#performance)
6. [Exercises](#exercises)
7. [Next Steps](#next)

---

## ⚙️ Setup

**Initialize Spark Session and sample data for practice.**

In [ ]:
// Import Spark libraries
import org.apache.spark.sql.SparkSession
import org.apache.spark.rdd.RDD

In [ ]:
// Create SparkSession
val spark = SparkSession.builder()
  .appName("RDD Operations Practice")
  .master("local[*]")
  .getOrCreate()

val sc = spark.sparkContext

println("🚀 RDD Operations Practice Session Started")
println(s"Available cores: ${sc.defaultParallelism}")

In [ ]:
// Sample datasets for practice
val numbers = sc.parallelize(Seq(1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
val words = sc.parallelize(Seq("hello", "world", "spark", "scala", "big", "data"))
val sales = sc.parallelize(Seq(
  ("Alice", 100.0),
  ("Bob", 150.0),
  ("Alice", 200.0),
  ("Charlie", 75.0),
  ("Bob", 120.0),
  ("Alice", 50.0)
))

println("📊 Sample datasets created:")
println(s"Numbers: ${numbers.collect().mkString("[", ", ", "]")}")
println(s"Words: ${words.collect().mkString("[", ", ", "]")}")
println(s"Sales: ${sales.collect().mkString("[", ", ", "]")}")

## 🔄 Essential Transformations

**Transformations create new RDDs from existing ones (lazy operations).**

In [ ]:
// map: Apply function to each element
println("🗺️ map Transformation:")
val doubled = numbers.map(_ * 2)
println(s"Original: ${numbers.collect().mkString(", ")}")
println(s"Doubled: ${doubled.collect().mkString(", ")}")

val upperWords = words.map(_.toUpperCase)
println(s"Words upper: ${upperWords.collect().mkString(", ")}")

In [ ]:
// filter: Keep elements matching predicate
println("\n🔍 filter Transformation:")
val evens = numbers.filter(_ % 2 == 0)
println(s"Even numbers: ${evens.collect().mkString(", ")}")

val longWords = words.filter(_.length > 4)
println(s"Long words (>4 chars): ${longWords.collect().mkString(", ")}")

val highSales = sales.filter(_._2 > 100)
println(s"High sales (>100): ${highSales.collect().mkString(", ")}")

In [ ]:
// flatMap: Apply function returning sequence, then flatten
println("\n📄 flatMap Transformation:")
val wordsFlat = words.flatMap(_.toCharArray)
println(s"Word characters: ${wordsFlat.collect().mkString(", ")}")

val sentences = sc.parallelize(Seq("Hello world", "Spark is awesome"))
val wordsFromSentences = sentences.flatMap(_.split(" "))
println(s"Words from sentences: ${wordsFromSentences.collect().mkString(", ")}")

val numbersAndSquares = numbers.flatMap(n => Seq(n, n*n))
println(s"Numbers and squares: ${numbersAndSquares.collect().mkString(", ")}")

In [ ]:
// distinct: Remove duplicates
println("\n🎯 distinct Transformation:")
val duplicates = sc.parallelize(Seq(1, 2, 2, 3, 3, 3, 4, 5, 5))
val unique = duplicates.distinct()
println(s"With duplicates: ${duplicates.collect().mkString(", ")}")
println(s"Distinct: ${unique.collect().mkString(", ")}")

val duplicateWords = sc.parallelize(Seq("hello", "world", "hello", "spark", "world"))
val uniqueWords = duplicateWords.distinct()
println(s"Words with duplicates: ${duplicateWords.collect().mkString(", ")}")
println(s"Unique words: ${uniqueWords.collect().mkString(", ")}")

In [ ]:
// union, intersection, subtract
println("\n🔗 Set Operations:")
val setA = sc.parallelize(Seq(1, 2, 3, 4, 5))
val setB = sc.parallelize(Seq(4, 5, 6, 7, 8))
val setC = sc.parallelize(Seq(1, 2, 6, 7, 9))

println(s"Set A: ${setA.collect().sorted.mkString(", ")}")
println(s"Set B: ${setB.collect().sorted.mkString(", ")}")
println(s"Set C: ${setC.collect().sorted.mkString(", ")}")

val unionAB = setA.union(setB)
println(s"A ∪ B: ${unionAB.collect().sorted.mkString(", ")}")

val intersectionAB = setA.intersection(setB)
println(s"A ∩ B: ${intersectionAB.collect().sorted.mkString(", ")}")

val subtractAC = setA.subtract(setC)
println(s"A - C: ${subtractAC.collect().sorted.mkString(", ")}")

## 📊 Aggregation Operations

**Actions that combine RDD elements into single values.**

In [ ]:
// reduce: Combine elements with associative function
println("🔢 reduce Action:")
val sum = numbers.reduce(_ + _)
println(s"Sum of numbers: $sum")

val product = numbers.filter(_ <= 5).reduce(_ * _)
println(s"Product of numbers ≤ 5: $product")

val concatenated = words.reduce(_ + " " + _)
println(s"Concatenated words: $concatenated")

// Note: reduce fails on empty RDD
// val emptySum = sc.parallelize(Seq[Int]()).reduce(_ + _)  // Throws exception

In [ ]:
// fold: Similar to reduce but with initial value (safer)
println("\n📦 fold Action:")
val sumWithFold = numbers.fold(0)(_ + _)
println(s"Sum with fold: $sumWithFold")

val emptySum = sc.parallelize(Seq[Int]()).fold(0)(_ + _)
println(s"Empty RDD sum: $emptySum")

val wordLengths = words.fold(0)(_ + _.length)
println(s"Total characters in words: $wordLengths")

val maxLength = words.fold(0)((acc, word) => math.max(acc, word.length))
println(s"Longest word length: $maxLength")

In [ ]:
// aggregate: Most general aggregation (initial value + combine functions)
println("\n🔧 aggregate Action:")

// Count and sum in one operation
val (count, total) = numbers.aggregate((0, 0))(
  // Within partition: (count, sum) + element
  (acc, num) => (acc._1 + 1, acc._2 + num),
  // Between partitions: combine results
  (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
)
println(f"Count: $count, Total: $total, Average: ${total.toDouble/count}%.2f")

// Statistics in one pass
case class Stats(count: Int, sum: Int, min: Int, max: Int)
val stats = numbers.aggregate(Stats(0, 0, Int.MaxValue, Int.MinValue))(
  (acc, num) => Stats(
    acc.count + 1,
    acc.sum + num,
    math.min(acc.min, num),
    math.max(acc.max, num)
  ),
  (acc1, acc2) => Stats(
    acc1.count + acc2.count,
    acc1.sum + acc2.sum,
    math.min(acc1.min, acc2.min),
    math.max(acc1.max, acc2.max)
  )
)
println(f"Stats - Count: ${stats.count}, Sum: ${stats.sum}, Min: ${stats.min}, Max: ${stats.max}, Avg: ${stats.sum.toDouble/stats.count}%.2f")

## 🔑 Key-Value Operations

**Operations on RDDs containing key-value pairs.**

In [ ]:
// groupByKey: Group values by key
println("🔑 groupByKey Operation:")
val groupedSales = sales.groupByKey()
println("Sales grouped by person:")
groupedSales.collect().foreach { case (person, amounts) =>
  println(f"  $person%-8s: ${amounts.mkString("[", ", ", "]")}")
}

// Warning: groupByKey can cause memory issues!
// Better to use reduceByKey when possible

In [ ]:
// reduceByKey: Combine values with same key (more efficient)
println("\n⚡ reduceByKey Operation:")
val totalSales = sales.reduceByKey(_ + _)
println("Total sales by person:")
totalSales.collect().foreach { case (person, total) =>
  println(f"  $person%-8s: $$${total}%.2f")
}

// Count words with reduceByKey
val wordCounts = words.map(word => (word, 1)).reduceByKey(_ + _)
println("\nWord counts:")
wordCounts.collect().foreach { case (word, count) =>
  println(f"  $word%-8s: $count")
}

In [ ]:
// Other key-value operations
println("\n🔄 Other Key-Value Operations:")

// sortByKey
val sortedSales = totalSales.sortByKey()
println("Sales sorted by name:")
sortedSales.collect().foreach { case (person, total) =>
  println(f"  $person%-8s: $$${total}%.2f")
}

// mapValues (transform values only)
val formattedSales = totalSales.mapValues(amount => f"$$$amount%.2f")
println("\nFormatted sales:")
formattedSales.collect().foreach { case (person, formatted) =>
  println(f"  $person%-8s: $formatted")
}

// keys and values
val people = totalSales.keys.distinct()
val amounts = totalSales.values
println(s"\nPeople: ${people.collect().mkString(", ")}")
println(s"Amounts: ${amounts.collect().mkString(", ")}")

In [ ]:
// join operations
println("\n🤝 Join Operations:")

val salesRDD = sc.parallelize(Seq(
  ("Alice", 100.0),
  ("Bob", 150.0),
  ("Charlie", 75.0)
))

val regionsRDD = sc.parallelize(Seq(
  ("Alice", "North"),
  ("Bob", "South"),
  ("David", "East")
))

val innerJoin = salesRDD.join(regionsRDD)
println("Inner join (sales with regions):")
innerJoin.collect().foreach { case (person, (sales, region)) =>
  println(f"  $person%-8s: $$$sales%.2f ($region)")
}

val leftJoin = salesRDD.leftOuterJoin(regionsRDD)
println("\nLeft join (all sales, optional regions):")
leftJoin.collect().foreach { case (person, (sales, regionOpt)) =>
  val region = regionOpt.getOrElse("Unknown")
  println(f"  $person%-8s: $$$sales%.2f ($region)")
}

val rightJoin = salesRDD.rightOuterJoin(regionsRDD)
println("\nRight join (all regions, optional sales):")
rightJoin.collect().foreach { case (person, (salesOpt, region)) =>
  val sales = salesOpt.map(s => f"$$$s%.2f").getOrElse("No sales")
  println(f"  $person%-8s: $sales ($region)")
}

## ⚡ Performance Patterns

**Understanding when to use each operation for optimal performance.**

In [ ]:
// Performance comparison: groupByKey vs reduceByKey
println("⚡ Performance Comparison:")

// Create larger dataset for meaningful comparison
val largeSales = sc.parallelize((1 to 1000).map(i => (s"Person${i % 10}", i.toDouble)))

println("Dataset size: " + largeSales.count())

// Method 1: groupByKey + mapValues (inefficient)
val start1 = System.nanoTime()
val result1 = largeSales.groupByKey().mapValues(_.sum).collect()
val time1 = (System.nanoTime() - start1) / 1e6

// Method 2: reduceByKey (efficient)
val start2 = System.nanoTime()
val result2 = largeSales.reduceByKey(_ + _).collect()
val time2 = (System.nanoTime() - start2) / 1e6

println(f"groupByKey + mapValues: ${time1}%.2f ms")
println(f"reduceByKey: ${time2}%.2f ms")
println(f"reduceByKey is ${time1/time2}%.1f x faster")

// Results should be the same
println("Results match: " + result1.toSet.equals(result2.toSet))

In [ ]:
// Understanding shuffles and data movement
println("\n🔀 Understanding Shuffles:")

// Operations that cause shuffles (data movement between nodes)
val data = sc.parallelize(Seq(1, 2, 3, 4, 5, 6, 7, 8, 9, 10))

println("Operations causing shuffles:")
println("- groupByKey: Moves all values for same key to one node")
println("- reduceByKey: Combines locally first, then moves")
println("- join: Requires keys to be on same node")
println("- sortByKey: Requires global ordering")
println("- distinct: Requires deduplication across nodes")

// Demonstrate reduceByKey vs groupByKey shuffle difference
val kvData = sc.parallelize(Seq(
  ("A", 1), ("B", 2), ("A", 3), ("B", 4), ("A", 5)
))

// reduceByKey: Combine locally before shuffle
val reduceResult = kvData.reduceByKey(_ + _)
println(s"\nreduceByKey result: ${reduceResult.collect().mkString(", ")}")

// groupByKey: Shuffle all values
val groupResult = kvData.groupByKey().mapValues(_.sum)
println(s"groupByKey result: ${groupResult.collect().mkString(", ")}")

println("\nBoth give same result, but reduceByKey is more efficient!")

## 🏆 Practice Exercises

**Apply what you've learned with these hands-on exercises.**

In [ ]:
// Exercise 1: Text Analysis
// Given a list of sentences, compute:
// - Total word count
// - Average words per sentence
// - Most frequent word
// - Words appearing in multiple sentences

val sentences = Seq(
  "Spark is a powerful big data framework",
  "Scala and Spark work great together",
  "Big data processing with Spark is efficient",
  "Spark provides excellent performance",
  "Learning Spark and Scala is rewarding"
)

// FIXME: Implement text analysis
val sentencesRDD = sc.parallelize(sentences)

// Total word count
val totalWords = ???  // Hint: flatMap + count

// Average words per sentence
val avgWordsPerSentence = ???  // Hint: map + reduce

// Most frequent word
val mostFrequentWord = ???  // Hint: flatMap + map + reduceByKey + sort

// Words in multiple sentences
val multiSentenceWords = ???  // Hint: flatMap + distinct + groupByKey + filter

println("📝 Text Analysis Results:")
println(s"Total words: $totalWords")
println(f"Average words per sentence: $avgWordsPerSentence%.1f")
println(s"Most frequent word: $mostFrequentWord")
println(s"Words in multiple sentences: ${multiSentenceWords.mkString(", ")}")

In [ ]:
// Exercise 2: Sales Analytics
// Given sales data, compute:
// - Total revenue by product
// - Top 3 products by revenue
// - Products with revenue > 500
// - Average revenue per product

val salesData = Seq(
  ("Laptop", 1200.0, 3),
  ("Mouse", 25.0, 10),
  ("Keyboard", 75.0, 5),
  ("Monitor", 300.0, 8),
  ("Laptop", 1200.0, 2),
  ("Mouse", 25.0, 15),
  ("Headphones", 150.0, 6),
  ("Monitor", 300.0, 4)
)

// FIXME: Implement sales analytics
val salesRDD = sc.parallelize(salesData)

// Calculate revenue (price * quantity) and group by product
val revenueByProduct = salesRDD.map { case (product, price, qty) =>
  (product, price * qty)
}.reduceByKey(_ + _)

// Total revenue by product
val totalRevenueByProduct = ???  // Hint: Use revenueByProduct

// Top 3 products by revenue
val top3Products = ???  // Hint: sortBy + take

// Products with revenue > 500
val highRevenueProducts = ???  // Hint: filter

// Average revenue per product
val avgRevenuePerProduct = ???  // Hint: Use revenueByProduct + aggregate

println("💰 Sales Analytics Results:")
println("Total revenue by product:")
totalRevenueByProduct.foreach { case (product, revenue) =>
  println(f"  $product%-12s: $$$revenue%.2f")
}
println(s"\nTop 3 products: ${top3Products.mkString(", ")}")
println(s"High revenue products (>500): ${highRevenueProducts.mkString(", ")}")
println(f"Average revenue per product: $$$avgRevenuePerProduct%.2f")

In [ ]:
// Exercise 3: Performance Optimization
// Compare different approaches for the same computation
// Goal: Find sum of squares for numbers 1-1000

val largeNumbers = 1 to 1000

// FIXME: Implement and compare different approaches
val numbersRDD = sc.parallelize(largeNumbers)

// Approach 1: map + reduce
val start1 = System.nanoTime()
val result1 = ???  // map(_ * _) then reduce(_ + _)
val time1 = (System.nanoTime() - start1) / 1e6

// Approach 2: aggregate
val start2 = System.nanoTime()
val result2 = ???  // Use aggregate with (0, 0) accumulator
val time2 = (System.nanoTime() - start2) / 1e6

// Approach 3: fold
val start3 = System.nanoTime()
val result3 = ???  // Use fold with 0
val time3 = (System.nanoTime() - start3) / 1e6

println("⚡ Performance Comparison - Sum of Squares (1-1000):")
println(f"map + reduce:     $time1%.2f ms (result: $result1)")
println(f"aggregate:         $time2%.2f ms (result: $result2)")
println(f"fold:             $time3%.2f ms (result: $result3)")

val fastest = List((time1, "map+reduce"), (time2, "aggregate"), (time3, "fold")).minBy(_._1)
println(s"\nFastest approach: ${fastest._2}")
println("All results match: " + (result1 == result2 && result2 == result3))

## 🛑 Cleanup

**Always stop your SparkSession to free resources!**

In [ ]:
// Stop SparkSession
spark.stop()
println("🛑 Spark Session Stopped")
println("✅ All resources cleaned up!")

## 📚 What Next?

**🎉 Congratulations!** You've mastered essential RDD operations!

**You've learned:**
- ✅ Core transformations: map, filter, flatMap, distinct
- ✅ Aggregation operations: reduce, fold, aggregate
- ✅ Key-value operations: groupByKey, reduceByKey, joins
- ✅ Set operations: union, intersection, subtract
- ✅ Performance patterns: choosing the right operation
- ✅ Lazy evaluation and execution optimization

**Key Concepts Mastered:**
- **Transformations vs Actions**: Lazy vs eager execution
- **Shuffle Operations**: Understanding data movement costs
- **Key-Value RDDs**: Efficient grouped operations
- **Performance Trade-offs**: When to use each operation
- **Distributed Computing**: How operations execute across nodes

**Next Steps:**
1. Complete all exercises with your own implementations
2. Experiment with different RDD operations on various datasets
3. Move to **03: DataFrame Operations** for structured data processing
4. Practice identifying which operations to use for different problems

**Performance Tip:** Always prefer `reduceByKey` over `groupByKey` + transformation for better performance!

**Remember:** RDDs are the foundation - everything in Spark builds on these core operations! ⚡